In [1]:
!pip install pandarallel
import pandas as pd
pd.set_option("max_colwidth", 600)
from bs4 import BeautifulSoup
import requests

from pandarallel import pandarallel


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for pandarallel: filename=pandarallel-1.6.1-py3-none-any.whl size=16265 sha256=ada4aa56f4fde90158ea5a595171abdd96cd09c6feda5ec4aae33f5183b32924
  Stored in directory: /root/.cache/pip/wheels/e8/81/8f/29aafd7d671a4e9db4e976d71728870173f0b3f48ccba0bc32
Successfully built pandarallel


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
def extract_urls_topic(url):
    try:
      coverpage = requests.get(url).content
      soup = BeautifulSoup(coverpage, 'html.parser')
     
      rows = []

      for link in soup.find_all('h3', class_='jeg_post_title'):   
        rows.append(link.a['href'])

    except:
      rows = 'error'
      
    return rows

In [5]:
extract_urls_topic('https://www.eldiario.net/portal/category/politica/page/193/')

['https://www.eldiario.net/portal/2021/05/10/continuan-cuestionamientos-por-impuesto-a-servicios-digitales/',
 'https://www.eldiario.net/portal/2021/05/10/jallalla-hara-conocer-acciones-contra-quispe/',
 'https://www.eldiario.net/portal/2022/07/18/profesor-edgar-barriga-esta-desaparecido-mas-de-10-dias/',
 'https://www.eldiario.net/portal/2022/07/18/dirigir-a-bolivia-es-un-desafio-muy-lindo/',
 'https://www.eldiario.net/portal/2022/07/17/cochabambina-fernanda-pavisic-es-miss-bolivia-universo-2022/',
 'https://www.eldiario.net/portal/2022/07/18/lusbel-309/',
 'https://www.eldiario.net/portal/2022/07/19/da-costa-pide-disculpas-sobre-jhasmani/',
 'https://www.eldiario.net/portal/2022/07/19/da-costa-pide-disculpas-sobre-jhasmani/',
 'https://www.eldiario.net/portal/2022/07/19/diecisiete-conductores-sancionados-por-conducir-en-estado-de-ebriedad/',
 'https://www.eldiario.net/portal/2022/07/19/mas-de-sesenta-barrios-del-d-7-piden-obras-a-la-alcaldia/',
 'https://www.eldiario.net/portal/2022/

In [7]:
pandarallel.initialize(nb_workers= 20, progress_bar=True)
df = pd.DataFrame(range(1,600),columns = ['page'])
url_str=''

#politica
for i in range(0,194) :
    url_str='https://www.eldiario.net/portal/category/politica/page/'+str(i)
    df.page[i] = url_str

#inter
for i in range(0,339) :
    url_str='https://www.eldiario.net/portal/category/internacional/page/'+str(i)
    df.page[i+194] = url_str

df['rows']=df['page'].parallel_apply(extract_urls_topic)
df = df[df.rows.str.len()!=0].reset_index(drop=True)
df = df.explode('rows', ignore_index=True)
df=df.drop_duplicates(subset='rows')
df = df.dropna()
print(df)
df.to_csv('/content/drive/MyDrive/Colab Notebooks/eldiario.lapaz/url.csv',index=False)

INFO: Pandarallel will run on 20 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


                                                                  page  \
0             https://www.eldiario.net/portal/category/politica/page/0   
1             https://www.eldiario.net/portal/category/politica/page/0   
2             https://www.eldiario.net/portal/category/politica/page/0   
3             https://www.eldiario.net/portal/category/politica/page/0   
4             https://www.eldiario.net/portal/category/politica/page/0   
...                                                                ...   
10635  https://www.eldiario.net/portal/category/internacional/page/338   
10636  https://www.eldiario.net/portal/category/internacional/page/338   
10637  https://www.eldiario.net/portal/category/internacional/page/338   
10638  https://www.eldiario.net/portal/category/internacional/page/338   
10649                                                              534   

                                                                                                               